In [ ]:
#Classify the speakers of given features.

In [2]:
import numpy as np
import torch
import random

def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    
set_seed(87)

In [18]:
import os
import json
import torch
import random
from pathlib import Path
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence

class myDataset(Dataset):
    def __init__(self, data_dir, segment_len=128):
        self.data_dir = data_dir
        self.segment_len = segment_len
        
        mapping_path = Path(data_dir) / "mapping.json"
        mapping = json.load(mapping_path.open())
        self.speaker2id = mapping["speaker2id"]
        
        metadata_path = Path(data_dir) / "metadata.json"
        metadata = json.load(open(metadata_path))["speakers"]
        
        self.speaker_num = len(metadata.keys())
        self.data = []
        for speaker in metadata.keys():
            for utterances in metadata[speaker]:
                self.data.append([utterances["feature_path"], self.speaker2id[speaker]])
                
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        feat_path,speaker = self.data[index]
        mel = torch.load(os.path.join(self.data_dir, feat_path))
        
        # Segmemt mel-spectrogram into "segment_len" frames.
        if len(mel) > self.segment_len:
            # Randomly get the starting point of the segment.
            # then Get a segment with "segment_len" frames.
            start = random.randint(0, len(mel) - self.segment_len)
            mel = torch.FloatTensor(mel[start:start+self.segment_len])
        else:
            mel = torch.FloatTensor(mel)
        # Turn the speaker id into long for computing loss 
        speaker = torch.FloatTensor([speaker]).long()
        return mel, speaker
    
    def get_speaker_number(self):
        return self.speaker_num

In [22]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence

def collate_batch(batch):
    mel, speaker = zip(*batch)
    # Because we train the model batch by batch, 
    # we need to pad the features in the same batch to make their lengths the same.
    # pad log 10^(-20) which is very small value.
    mel = pad_sequence(mel, batch_first=True, padding_value=-20)
    return mel, torch.FloatTensor(speaker).long()

def get_dataloader(data_dir, batch_size, n_workers):
    dataset = myDataset(data_dir)
    speaker_num = dataset.get_speaker_number()
    trainlen = int(0.9 * len(dataset))
    lengths = [trainlen, len(dataset) - trainlen]
    trainset, validset = random_split(dataset, lengths)
    
    train_loader = DataLoader(
        trainset,
        batch_size=batch_size,
        shuffle=True,
        drop_last=True,
        num_workers=n_workers,
        pin_memory=True,
        collate_fn=collate_batch
    )
    valid_loader = DataLoader(
        validset,
        batch_size=batch_size,
        drop_last=True,
        num_workers=n_workers,
        pin_memory=True,
        collate_fn=collate_batch
    )
    
    return train_loader, valid_loader, speaker_num

In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Classifier(nn.Module):
    def __init__(self, d_model=80, n_spks=600, dropout=0.1):
        super().__init__()
        # Project the dimension of features from that of input into d_model.
        self.prenet = nn.Linear(40, d_model)
        
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, dim_feedforward=256, nhead=2
        )
        
        # Project the the dimension of features from d_model into speaker nums.
        self.pred_layer = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.ReLU(),
            nn.Linear(d_model, n_spks)
        )
        
    def forward(self, mels):
        """
        args:
            mels: (batch size, length, 40)
        return:
            out: (batch size, n_spks)
        """
        out = self.prenet(mels) #(batch size, length, d_model)
        out = out.permute(1, 0, 2) #(length, batch size, d_model)
        out = self.encoder_layer(out) #(length, batch size, d_model)
        out = out.transpose(0, 1) #(batch size, length, d_model)
        stats = out.mean(dim=1) #(batch, d_model)
        out = self.pred_layer(stats) #(batch, n_spks)
        return out

In [8]:
#warmup
#Set learning rate to 0 in the beginning.
#The learning rate increases linearly from 0 to initial learning rate during warmup period.

import math
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR

# 设置学习率为optimizer中学习率Xlr_lambda函数的值
def get_cosine_schedule_with_warmup(
    optimizer: Optimizer,
    num_warmup_steps: int,
    num_training_steps: int,
    num_cycles: float = 0.5,
    last_epoch: int = -1,
    ):
    """
    Create a schedule with a learning rate that decreases following the values of the cosine function between the
    initial lr set in the optimizer to 0, after a warmup period during which it increases linearly between 0 and the
    initial lr set in the optimizer.

    Args:
        optimizer (:class:`~torch.optim.Optimizer`):
        The optimizer for which to schedule the learning rate.
        num_warmup_steps (:obj:`int`):
        The number of steps for the warmup phase.
        num_training_steps (:obj:`int`):
        The total number of training steps.
        num_cycles (:obj:`float`, `optional`, defaults to 0.5):
        The number of waves in the cosine schedule (the defaults is to just decrease from the max value to 0
        following a half-cosine).
        last_epoch (:obj:`int`, `optional`, defaults to -1):
        The index of the last epoch when resuming training.

    Return:
        :obj:`torch.optim.lr_scheduler.LambdaLR` with the appropriate schedule.
    """
    def lr_lambda(current_step):
        # Warmup
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        # decadence
        progress = float(current_step - num_warmup_steps) / float(
            max(1, num_training_steps - num_warmup_steps)
        )
        return max(
            0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))
        )

    return LambdaLR(optimizer, lr_lambda, last_epoch)

In [29]:
def model_fn(batch, model, criterion, device):
    mels, labels = batch
    mels = mels.to(device)
    labels = labels.to(device)
    
    outs = model(mels)
    
    loss = criterion(outs, labels)
    
    preds = outs.argmax(1)
    accuracy = torch.mean((preds==labels).float())
    
    return loss, accuracy

In [10]:
from tqdm import tqdm

def valid(dataloader, model, criterion, device):
    model.eval()
    running_loss = 0.0
    running_accuracy = 0.0
    pbar = tqdm(total=len(dataloader.dataset), ncols=0, desc="Valid", unit="uttr")
    
    for i, batch in enumerate(dataloader):
        with torch.no_grad():
            loss, accuracy = model_fn(batch, model, criterion, device)
            running_loss += loss.item()
            running_accuracy += accuracy.item()
        pbar.update(dataloader.batch_size)
        pbar.set_postfix(
            loss=f"{running_loss / (i+1):.2f}",
            accuracy=f"{running_accuracy / (i+1):.2f}",
        )
    pbar.close()
    model.train()
    
    return running_accuracy / len(dataloader)
        

In [ ]:
from torch.optim import AdamW

def parse_args():
    """arguments"""
    config = {
        "data_dir": "/kaggle/input/ml2022spring-hw4/Dataset",
        "save_path": "model.ckpt",
        "batch_size": 32,
        "n_workers": 8,
        "valid_steps": 2000,
        "warmup_steps": 1000,
        "save_steps": 10000,
        "total_steps": 70000,
    }

    return config

def main(
    data_dir,
    save_path,
    batch_size,
    n_workers,
    valid_steps,
    warmup_steps,
    total_steps,
    save_steps,
    ):
    """Main function."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"[Info]: Use {device} now!")
    
    train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
    train_iterator = iter(train_loader)
    print(f"[Info]: Finish loading data!",flush = True)
    
    model = Classifier(n_spks=speaker_num).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=1e-3)
    scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
    print(f"[Info]: Finish creating model!",flush = True)
    
    best_accuracy = -1.0
    best_state_dict=  None
    
    pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")
    
    for step in range(total_steps):
        try:
            batch = next(train_iterator)
        except StopIteration:
            train_iterator = iter(train_loader)
            batch = next(train_iterator)
        
        loss, accuracy = model_fn(batch, model, criterion, device)
        batch_loss = loss.item()
        batch_accuracy = accuracy.item()
        
        
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        
        pbar.update()
        pbar.set_postfix(
            loss=f"{batch_loss:.2f}",
            accuracy=f"{batch_accuracy:.2f}",
            step=step + 1,
        )
        
        if (step+1) % valid_steps == 0:
            pbar.close()
            
            valid_accuracy = valid(valid_loader, model, criterion, device)
            
            if valid_accuracy > best_accuracy:
                best_accuracy = valid_accuracy
                best_state_dict = model.state_dict()
            pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit="step")
        
        if (step+1) % save_steps == 0 and best_state_dict is not None:
            torch.save(best_state_dict, save_path)
            pbar.write(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")
            
        pbar.close()
        
if __name__ == "__main__":
    main(**parse_args())

[Info]: Use cuda now!


/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[Info]: Finish loading data!
[Info]: Finish creating model!




Train:   0% 0/2000 [00:00<?, ? step/s]

Train:   0% 1/2000 [00:00<16:57,  1.96 step/s]

Train:   0% 1/2000 [00:00<17:27,  1.91 step/s, accuracy=0.00, loss=6.42, step=1]


Valid:   0% 0/5667 [00:00<?, ?uttr/s]

Valid:   1% 32/5667 [00:00<02:03, 45.76uttr/s]

Valid:   1% 32/5667 [00:00<02:03, 45.76uttr/s, accuracy=0.22, loss=3.89]

Valid:   1% 64/5667 [00:00<02:02, 45.76uttr/s, accuracy=0.16, loss=4.01]

Valid:   2% 96/5667 [00:00<02:01, 45.76uttr/s, accuracy=0.17, loss=4.00]

Valid:   2% 128/5667 [00:00<02:01, 45.76uttr/s, accuracy=0.15, loss=3.97]

Valid:   3% 160/5667 [00:00<00:22, 247.10uttr/s, accuracy=0.15, loss=3.97]

Valid:   3% 160/5667 [00:00<00:22, 247.10uttr/s, accuracy=0.15, loss=4.01]

Valid:   3% 192/5667 [00:00<00:22, 247.10uttr/s, accuracy=0.17, loss=3.98]

Valid:   4% 224/5667 [00:00<00:22, 247.10uttr/s, accuracy=0.18, loss=3.93]

Valid:   5% 256/5667 [00:00<00:21, 247.10uttr/s, accuracy=0.18, loss=3.89]

Valid:   5% 288/5667 [00:01<00:17, 307.05uttr/s, accuracy=0.18,

Step 10000, best model saved. (accuracy=0.4121)




Valid:   0% 0/5667 [00:00<?, ?uttr/s]

Valid:   1% 32/5667 [00:00<01:23, 67.76uttr/s]

Valid:   1% 32/5667 [00:00<01:23, 67.76uttr/s, accuracy=0.41, loss=2.39]

Valid:   1% 64/5667 [00:00<01:22, 67.76uttr/s, accuracy=0.45, loss=2.44]

Valid:   2% 96/5667 [00:00<01:22, 67.76uttr/s, accuracy=0.44, loss=2.38]

Valid:   2% 128/5667 [00:00<01:21, 67.76uttr/s, accuracy=0.45, loss=2.37]

Valid:   3% 160/5667 [00:00<00:16, 344.02uttr/s, accuracy=0.45, loss=2.37]

Valid:   3% 160/5667 [00:00<00:16, 344.02uttr/s, accuracy=0.47, loss=2.34]

Valid:   3% 192/5667 [00:00<00:15, 344.02uttr/s, accuracy=0.47, loss=2.36]

Valid:   4% 224/5667 [00:00<00:15, 344.02uttr/s, accuracy=0.46, loss=2.41]

Valid:   5% 256/5667 [00:00<00:15, 344.02uttr/s, accuracy=0.46, loss=2.38]

Valid:   5% 288/5667 [00:00<00:15, 344.02uttr/s, accuracy=0.45, loss=2.44]

Valid:   6% 320/5667 [00:00<00:08, 615.97uttr/s, accuracy=0.45, loss=2.44]

Valid:   6% 320/5667 [00:00<00:08, 615.97uttr/s, accuracy=0.45, loss=2.53]

Valid: